<a href="https://colab.research.google.com/github/kaniska/covid-19-hackathon/blob/master/social-network-mining/MedicalEntityRecognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%%time
!pip install pybind11

In [0]:
%%time
!pip install nmslib

In [0]:
%%time
!pip install scispacy spacy

In [0]:
!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.2.4/en_core_sci_sm-0.2.4.tar.gz

In [0]:
!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.2.4/en_core_sci_md-0.2.4.tar.gz

In [0]:
!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.2.4/en_ner_bc5cdr_md-0.2.4.tar.gz

In [0]:
import spacy
import scispacy
from scispacy.abbreviation import AbbreviationDetector
from scispacy.umls_linking import UmlsEntityLinker

In [0]:
#import en_core_web_sm
#import en_core_sci_sm
import en_ner_bc5cdr_md
#nlp1 = en_core_web_sm.load()
#nlp2 = en_core_sci_sm.load()
nlp3 = en_ner_bc5cdr_md.load()

linker = UmlsEntityLinker(resolve_abbreviations=True)
#nlp2.add_pipe(linker)

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
nlp3.add_pipe(linker)

In [0]:
!pip install pytextrank

In [0]:
import pytextrank
textRank = pytextrank.TextRank()
abbreviation_pipe = AbbreviationDetector(nlp3)
if not nlp3.has_pipe("abbrev"):
    nlp3.add_pipe(abbreviation_pipe, name="abbrev")
if not nlp3.has_pipe("textRank"):
    nlp3.add_pipe(textRank.PipelineComponent, name="textRank", last=True)
if not nlp3.has_pipe("linker"):
    nlp3.add_pipe(linker, name="linker")

In [0]:
import json
from collections import defaultdict, Counter

with open(r"/content/drive/My Drive/TweetStreams/coronavirus-tweet-id-2020-04-10-21.jsonl", "r") as json_file:
    json_list = list(json_file)
#tweets_data = ""
i = 0
#processed = []

for json_str in json_list:
    i= i+1
    
    tweet_data = json.loads(json_str)
        
    json_str = tweet_data['full_text']
    if "covid" not in json_str:
        continue
    #Approach 1
    doc1 = nlp3(json_str)
    #processed.append((tweet_data['id'], doc1))
    drugs = []
    diseases = []
    umls_entities = defaultdict(Counter)
    entities = defaultdict(lambda: defaultdict(Counter))

    
    entity_found = 0
    for entity in doc1.ents:
        # first find the entities and check the categories
        # 
        if entity.label_  in ['CHEMICAL', 'DISEASE']:
            print("Tweet: ",json_str)
            print("Name: ", entity.text, "Label: ", entity.label_)
            entities[tweet_data['id']][entity.label_].update([entity])
            for umls_ent in entity._.umls_ents:
                print(linker.umls.cui_to_entity[umls_ent[0]])
                print(umls_ent[1])
                print("---------------------------------------------")
            # the umls candidate entities are attached to the entity mention, we'll take only the first
            top_candidate = next(iter([e for e in entity._.umls_ents if e[1] > 0.85]), None)
            if top_candidate:
                cuid = top_candidate[0]
                umls_entities[tweet_data['id']].update([cuid])
                entity_found = 1
        else:
            print("Medical term not found")

    for cuid, freq in umls_entities[tweet_data['id']].most_common():
        entity = linker.umls.cui_to_entity[cuid]
        print(f"{freq} UID: {entity.concept_id}, name: {entity.canonical_name}, types: {entity.types}")

    
    if entity_found == 1:
        umls_entities[tweet_data['id']]
        entities[tweet_data['id']]
        break

    #Approach 2
    '''
    doc2 = nlp2(json_str)
    drugs = []
    diseases = []
    for entity in doc2.ents:
        print("---------------------------------------------")
        print("Name: ", entity.text, "Label: ", entity.label)
        print("---------------------------------------------")
        for umls_ent in entity._.umls_ents:
            print(linker.umls.cui_to_entity[umls_ent[0]])
            print("---------------------------------------------")
     
    break
    '''
    #tweets_data = tweets_data + json_str 
    #result = json.loads(json_str)
    #print("result: {}".format(result))
    #tweets_data = tweets_data + result 


In [0]:
for entity in doc.ents:
        entity_details = entity.text
        label = entity.label_
        if label == "CHEMICAL":
            drugs.append(entity_details)
        elif label == "DISEASE":
            diseases.append(entity_details)
    
    print(drugs)
    print(diseases)

In [0]:

#print(tweets_data)

doc = nlp1(tweets_data)
drugs = []
ds = []
diseases = []
for entity in doc.ents:
    drug = entity.text
    label = entity.label_
    if label == "CHEMICAL" and drug not in drugs:
        diseases.append(get_disease(drug))
    elif label == "DISEASE" and drug not in drugs:
        ds.append(drug)
    drugs.append(drug)
if len(ds):
    diseases.append(ds)
if len(diseases) == 1:
    print(None)
else:
    return print(diseases)

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
import argparse
import io
import json
import os

from google.cloud import language
import numpy
import six

from google.cloud.language import enums
from google.cloud.language import types
import os
from google.oauth2 import service_account
import googletrans
from googletrans import Translator
translator = Translator()
language_client = language.LanguageServiceClient.from_service_account_json("/content/drive/My Drive/secrets/niagra-a628a699be36.json")

#text = "Mulan Inspired merchandise available in China at Disney Store / Disney Town https://t.co/ZniMSngtwC"
#text = "RT @JesusCintora: Nos hablan de que los socialcomunistas han propagado el covid, de una gestión criminal o de campos de concentración, pero…"
result = translator.translate('RT @JesusCintora: Nos hablan de que los socialcomunistas han propagado el covid, de una gestión criminal o de campos de concentración, pero')
text = result.text

document = language.types.Document(
    content=text,
    type=language.enums.Document.Type.PLAIN_TEXT)
response1 = language_client.analyze_entities(document)
show_entities(response1,5)

response2 = language_client.classify_text(document)
show_topic(response2)

#print(response)




In [0]:
def show_topic(response):
    for category in response.categories:
        # Get the name of the category representing the document.
        # See the predefined taxonomy of categories:
        # https://cloud.google.com/natural-language/docs/categories
        print(u"Category name: {}".format(category.name))
        # Get the confidence. Number representing how certain the classifier
        # is that this category represents the provided text.
        print(u"Confidence: {}".format(category.confidence))



In [0]:
def show_entities(entities, n):
	for entity in entities.entities[:n]:
		print('=' * 20)
		print(' name: {0}'.format(entity.name))
		print(' type: {0}'.format(entity.Type.Name(entity.type)))
		print(' metadata: {0}'.format(dict(entity.metadata)))
		print(' salience: {:.4f}'.format(entity.salience))

In [0]:
!pip install googletrans

In [0]:
import googletrans
from googletrans import Translator
translator = Translator()
result = translator.translate('RT @JesusCintora: Nos hablan de que los socialcomunistas han propagado el covid, de una gestión criminal o de campos de concentración, pero')
print(result)